In [ ]:
%%capture
import ROOT
import glob
import math
import sys
import numpy as np
import pandas as pd
from IPython.display import display, Markdown, HTML
import ipywidgets as widgets
from TPCQCVis.src.drawHistograms import *
from TPCQCVis.src.drawTrending import *
from TPCQCVis.src.drawMultiTrending import *
from TPCQCVis.src.checkHistograms import *
from TPCQCVis.src.checkTrending import *
from TPCQCVis.src.utility import *
import warnings
from copy import copy
warnings.filterwarnings('ignore')

In [ ]:
%jsroot on
display(HTML("<style>.container { width:95% !important; align-items: center;}</style>"))
display(HTML("<style>table {float:left;}</style>"))
ROOT.gErrorIgnoreLevel = ROOT.kFatal
#display(HTML('<style>{}</style>'.format(CSS)))
#ROOT.gStyle.SetPalette(57)

In [ ]:
def drawQuantileProjection(objectName,rootDataFile,quantileOrder=0.5, axis="x",size=[600,400]):
    from copy import copy
    canvas = ROOT.TCanvas("","",size[0],size[1])
    legend = ROOT.TLegend()
    [hist,leg,canvo,pad1] = drawHistograms(objectName,rootDataFile,normalize=False,legend=False,
                                               legendNames=runList,pads=True,drawOption="COLZ",maxColumns=3)
    quants = []
    canvas.cd()
    ROOT.gPad.SetGridy(1)
    for i,histo in enumerate(hist):
        if axis == "y":
            quant = copy(histo.QuantilesY(quantileOrder))
        else:
            quant = copy(histo.QuantilesX(quantileOrder))
        quant.SetYTitle("Median "+ histo.GetYaxis().GetTitle())
        quant.SetXTitle(histo.GetXaxis().GetTitle())
        quant.SetTitle(quant.GetYaxis().GetTitle()+" vs. "+quant.GetXaxis().GetTitle())
        quant.SetLineWidth(2)
        quants.append(quant)
        quants[i].SetStats(0)
        quants[i].Draw("HIST E SAME")
        #legend.AddEntry(quant,runList[i])
        legend.AddEntry(quant,LegRunParam[i])
    legend.SetNColumns(math.ceil(len(quants)/8))
    legend.SetHeader("#Run polarity IRstart IRmid IRend", "C")
    return quants,legend,canvas


def drawProjection(objectName,rootDataFile,quantileOrder=0.5, axis="x",size=[600,400]):
    from copy import copy
    canvas = ROOT.TCanvas("","",size[0],size[1])
    legend = ROOT.TLegend()
    [hist,leg,canvo,pad1] = drawHistograms(objectName,rootDataFile,normalize=False,legend=False,
                                               legendNames=runList,pads=True,drawOption="COLZ",maxColumns=3)
    quants = []
    canvas.cd()
    ROOT.gPad.SetGridy(1)
    for i,histo in enumerate(hist):
        if axis == "y":
            quant = copy(histo.ProjectionY())
        else:
            quant = copy(histo.ProjectionX())
        #quant.SetYTitle("Median "+ histo.GetYaxis().GetTitle())
        #quant.SetXTitle(histo.GetXaxis().GetTitle())
        #quant.SetTitle(quant.GetYaxis().GetTitle()+" vs. "+quant.GetXaxis().GetTitle())
        quant.SetLineWidth(2)
        quants.append(quant)
        quants[i].SetStats(0)
        quants[i].Draw("HIST E SAME")
        #legend.AddEntry(quant,runList[i])
        legend.AddEntry(quant,LegRunParam[i])
    legend.SetNColumns(math.ceil(len(quants)/8))
    legend.SetHeader("#Run polarity IRstart IRmid IRend", "C")
    return quants,legend,canvas

def drawProfile(objectName,rootDataFile,quantileOrder=0.5, axis="x",size=[600,400]):
    from copy import copy
    canvas = ROOT.TCanvas("","",size[0],size[1])
    legend = ROOT.TLegend()
    [hist,leg,canvo,pad1] = drawHistograms(objectName,rootDataFile,normalize=False,legend=False,
                                               legendNames=runList,pads=True,drawOption="COLZ",maxColumns=3)
    quants = []
    canvas.cd()
    ROOT.gPad.SetGridy(1)
    for i,histo in enumerate(hist):
        if axis == "y":
            quant = copy(histo.ProfileY())
        else:
            quant = copy(histo.ProfileX())
        #quant.SetYTitle("Median "+ histo.GetYaxis().GetTitle())
        #quant.SetXTitle(histo.GetXaxis().GetTitle())
        #quant.SetTitle(quant.GetYaxis().GetTitle()+" vs. "+quant.GetXaxis().GetTitle())
        quant.SetLineWidth(2)
        quants.append(quant)
        quants[i].SetStats(0)
        quants[i].Draw("HIST E SAME")
        #legend.AddEntry(quant,runList[i])
        legend.AddEntry(quant,LegRunParam[i])
    legend.SetNColumns(math.ceil(len(quants)/8))
    legend.SetHeader("#Run polarity IRstart IRmid IRend", "C")
    return quants,legend,canvas

    
def sliceAndFit(objectName,rootDataFile,fitFunc="gaus",fitRange=[40,60],size=[600,400]):
    from copy import copy
    canvas = ROOT.TCanvas("","",size[0],size[1])
    legend = ROOT.TLegend()
    [hist,leg,canvo,pad1] = drawHistograms(objectName,rootDataFile,normalize=False,legend=False,
                                               legendNames=runList,pads=True,drawOption="COLZ",maxColumns=3)
    
    fits = []
    myFunc = ROOT.TF1("myFunc","gaus",40,60)
    canvas.cd()
    for i,histo in enumerate(hist):
        try:
            histo.FitSlicesY(myFunc, 0, -1, 0, "QNR")
            fit = copy(ROOT.gDirectory.Get(histo.GetName()+"_1"))
        except:
            print("Could not perform fit.")
            return [],legend,canvas
        fit.SetYTitle("Gaus Fit Mean "+ histo.GetYaxis().GetTitle())
        fit.SetXTitle(histo.GetXaxis().GetTitle())
        fit.SetTitle(histo.GetYaxis().GetTitle()+" vs. "+histo.GetXaxis().GetTitle())
        fit.SetLineWidth(2)
        legend.AddEntry(fit, LegRunParam[i])
        fits.append(fit)
        fit.Draw("HIST E SAME")
    legend.SetNColumns(math.ceil(len(hist)/8))
    legend.SetHeader("#Run polarity IRstart IRmid IRend", "C")
    legend.Draw()
    return fits, legend, canvas

def sliceAndFitWidth(objectName,rootDataFile,fitFunc="gaus",fitRange=[40,60],size=[600,400]):
    from copy import copy
    canvas = ROOT.TCanvas("","",size[0],size[1])
    legend = ROOT.TLegend()
    [hist,leg,canvo,pad1] = drawHistograms(objectName,rootDataFile,normalize=False,legend=False,
                                               legendNames=runList,pads=True,drawOption="COLZ",maxColumns=3)
    
    fits = []
    myFunc = ROOT.TF1("myFunc","gaus",40,60)
    canvas.cd()
    for i,histo in enumerate(hist):
        try:
            histo.FitSlicesY(myFunc, 0, -1, 0, "QNR")
            fit = copy(ROOT.gDirectory.Get(histo.GetName()+"_2"))
        except:
            print("Could not perform fit.")
            return [],legend,canvas
        fit.SetYTitle("Gaus Fit Width "+ histo.GetYaxis().GetTitle())
        fit.SetXTitle(histo.GetXaxis().GetTitle())
        fit.SetTitle(histo.GetYaxis().GetTitle()+" vs. "+histo.GetXaxis().GetTitle())
        fit.SetLineWidth(2)
        legend.AddEntry(fit,LegRunParam[i])
        fits.append(fit)
        fit.Draw("HIST E SAME")
    legend.SetNColumns(math.ceil(len(hist)/8))
    legend.SetHeader("#Run polarity IRstart IRmid IRend", "C")
    legend.Draw()
    return fits, legend, canvas

In [ ]:
ROOT.gStyle.SetPalette(57)
ROOT.gStyle.SetPalette(55)
ROOT.gStyle.SetGridStyle(3)
ROOT.gStyle.SetGridWidth(1)
ROOT.gStyle.SetOptStat(0)
cols = list(ROOT.TColor.GetPalette())
def updateColors(histograms,palette):
    colors = []
    for i,hist in enumerate(histograms):
        if len(histograms)>1:
            color = palette[math.floor((i/(len(histograms)-1))*(len(palette)-1))]
        else:
            color = palette[0]
        #color = i
        colors.append(color)
        hist.SetLineColor(color)
        hist.SetMarkerColor(color)
    return colors

def calculate_kl_divergence(P, Q):
    epsilon = 1e-10  # A small value to avoid division by zero
    kl_divergence = 0
    for p, q in zip(P, Q):
        # Only calculate if both p and q are greater than a very small value
        if p > epsilon and q > epsilon:
            kl_divergence += p * math.log(p / q)
        # If q is too close to zero, we skip the calculation for this term
        #elif q <= epsilon:
            #print(f"Warning: q is too close to zero, skipping the term (p={p}, q={q})")
    return kl_divergence

def normalize_distribution(histogram):
    total = sum(histogram)
    return [x / total for x in histogram] if total > 0 else histogram

def outlierDetection(histogram_name, histograms, runList, draw_plot=False):
    """
    Detects the outlier run based on KL divergence from the mean distribution.

    This function calculates the KL divergence between each distribution in 'histograms' 
    and the mean of all distributions. It identifies the run with the highest KL 
    divergence, which is considered the most divergent (outlier) distribution.

    Parameters:
    histogram_name (str): The name of the histogram, used as the title of the KL divergence plot.
    histograms (list): A list of distributions to compare against the mean.
    runList (list): A list of run numbers corresponding to the distributions in 'histograms'.
    draw_plot (bool): If True, draws the KL divergence plot. If False, only prints the outlier information.

    Returns:
    hOutlier (ROOT.TH1F): The histogram of KL divergence values for each run (if draw_plot=True).
    cOutlier (ROOT.TCanvas): The canvas object containing the KL divergence plot (if draw_plot=True).
    
    The function will print the run number that has the highest KL divergence, indicating 
    the most divergent (outlier) run.
    """
    files = len(runList)

    # Creating the canvas and histogram objects only if draw_plot is True
    if draw_plot:
        # Outlier canvas
        cOutlier = ROOT.TCanvas("cOutlier", "KL divergence", 1100, 400)
        cOutlier.SetBottomMargin(0.15)
        cOutlier.SetGridx()

        # Outlier distribution
        hOutlier = ROOT.TH1F("h_kl_divergence", histogram_name + ";Run;KL divergence", 
                             files, 0, files)
        hOutlier.SetLineWidth(2)
        hOutlier.SetMarkerSize(1.5)
        hOutlier.SetMarkerStyle(ROOT.kFullCircle)
        hOutlier.SetStats(0)

    # Building mean histogram
    meanHistogram = None
    for i,histogram in enumerate(histograms):
        if i == 0:
            meanHistogram = histogram.Clone("meanHistogram")
        else:
            meanHistogram.Add(histogram)
        
        if i == files - 1:
            meanHistogram.Scale(1 / len(histograms))

    kl_divergence_values = []
    max_kl_run_number = -1
    max_kl_divergence = - math.inf # starting with very low value to be compared

    for i in range(len(histograms)):
        kl_divergence_value = calculate_kl_divergence(normalize_distribution(histograms[i]), normalize_distribution(meanHistogram))
        kl_divergence_values.append(kl_divergence_value)
        if draw_plot:
            hOutlier.Fill(i, kl_divergence_value)
            hOutlier.GetXaxis().SetBinLabel(i + 1, runList[i])
        if kl_divergence_value > max_kl_divergence:  # Change this condition
            max_kl_divergence = kl_divergence_value
            max_kl_run_number = int(runList[i])  # Store the corresponding run number as an integer (convert from string)
    
    # Outlier: the biggest kl value is the most divergent distribution
    if max_kl_run_number > -1:
        print(f"The most divergent run is: {max_kl_run_number} with a KL divergence of {max_kl_divergence}")
    
    # Draw the plot only if draw_plot is True
    if draw_plot:
        # Auto-adjust Y-axis to fit all KL divergence points
        min_value = min(kl_divergence_values)
        max_value = max(kl_divergence_values)
        padding = 0.1 * (max_value - min_value)
        hOutlier.GetYaxis().SetRangeUser(min_value - padding, max_value + padding)  # Add 10% padding
        hOutlier.LabelsOption("v")
        # Draw the KL values distribution
        #hOutlier.Draw("SAME L P E")
        hOutlier.Draw("HIST E SAME")
        cOutlier.Update()

        return hOutlier, cOutlier

    # Return None if plot is not drawn
    return None, None

In [ ]:
# Notebook variables
periodName = "myPeriod"
passName = "myPass"
runNumber = 123456
path = "myPath"

#periodName = "LHC23m"
#passName = "apass3"
#runNumber = "536545"
#path = "/cave/alice-tpc-qc/data/2023/"

# periodName = "LHC23zzk"
# passName = "apass4"
# runNumber = "544451"
# path = "/home/ferrandi/alice/TPCQCVis/TPCQCVis/DownloadedData/2023"

# periodName = "LHC24af"
# passName = "apass1"
# runNumber = "550367"
# path = "/home/ferrandi/alice/TPCQCVis/TPCQCVis/DownloadedData/2024"

# Read the Root Files
fileList = glob.glob(path+"/"+periodName+"/"+passName+"/"+"*_QC.root")
fileList = [file for file in fileList if file[-13] != "_"]
fileList.sort()
#fileList = fileList[13:]
runList = [fileList[i][-14:-8] for i in range(len(fileList))]
rootDataFile=[]
for file in fileList:
    rootDataFile.append(ROOT.TFile.Open(file,"READ"))
    
colors = updateColors([ROOT.TH1C("","",0,0,0) for _ in range(len(rootDataFile))],cols)

# TPC Async QC Report - myPeriod - myPass
**[RCT Table](https://ali-bookkeeping.cern.ch/?page=lhc-period-overview)**

In [ ]:
import datetime
now = datetime.datetime.now()
display(Markdown("Report automatically created on **"+now.strftime("%d.%m.%Y @ %H:%M:%S")+"**"))

In [ ]:
runList_markdown = ""
for run in runList:
    runList_markdown += f"`{run}` "

display(Markdown("**Runs:** \n\n"+runList_markdown))

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Run-Parameters"data-toc-modified-id="Run-Parameters-1">Run Parameters</a></span></li></ul><ul class="toc-item"><li><span><a href="#Quality-Observer" data-toc-modified-id="Quality-Observer-1">Quality Observer</a></span><ul class="toc-item"><li><span><a href="#Clusters" data-toc-modified-id="Clusters-1.1">Clusters</a></span></li><li><span><a href="#Tracks" data-toc-modified-id="Tracks-1.2">Tracks</a></span></li></ul></li><li><span><a href="#Trendings" data-toc-modified-id="Trendings-2">Trendings</a></span><ul class="toc-item"><li><span><a href="#Mean-phi-of-tracks" data-toc-modified-id="Mean-phi-of-tracks-2.1">Mean phi of tracks</a></span></li><li><span><a href="#Mean-eta-of-tracks" data-toc-modified-id="Mean-eta-of-tracks-2.2">Mean eta of tracks</a></span></li><li><span><a href="#Mean-tracks-transverse-momentum-of-tracks" data-toc-modified-id="Mean-tracks-transverse-momentum-of-tracks-2.3">Mean tracks transverse momentum of tracks</a></span></li><li><span><a href="#Mean-charge-sign-of-tracks" data-toc-modified-id="Mean-charge-sign-of-tracks-2.4">Mean charge sign of tracks</a></span></li><li><span><a href="#Mean-dEdxTot-of-MIPs-tracks" data-toc-modified-id="Mean-dEdxTot-of-MIPs-tracks-2.5">Mean dEdxTot of MIPs tracks</a></span></li></ul></li><li><span><a href="#Plots" data-toc-modified-id="Plots-3">Plots</a></span><ul class="toc-item"><li><span><a href="#Clusters" data-toc-modified-id="Clusters-3.1">Clusters</a></span></li><li><span><a href="#Tracks" data-toc-modified-id="Tracks-3.2">Tracks</a></span><ul class="toc-item"><li><span><a href="#Geometrical-distributions-of-tracks" data-toc-modified-id="Geometrical-distributions-of-tracks-3.2.1">Geometrical distributions of tracks</a></span></li><li><span><a href="#Track-properties" data-toc-modified-id="Track-properties-3.2.2">Track properties</a></span></li></ul></li><li><span><a href="#DCAr" data-toc-modified-id="DCAr-3.3">DCAr</a></span><ul class="toc-item"><li><span><a href="#DCAr-vs-phi" data-toc-modified-id="DCAr-vs-phi-3.3.1">DCAr vs phi</a></span></li><li><span><a href="#DCAr-vs-eta" data-toc-modified-id="DCAr-vs-eta-3.3.2">DCAr vs eta</a></span></li><li><span><a href="#DCAr-vs-pT" data-toc-modified-id="DCAr-vs-pT-3.3.3">DCAr vs pT</a></span></li><li><span><a href="#DCAr-vs-nClusters" data-toc-modified-id="DCAr-vs-nClusters-3.3.4">DCAr vs nClusters</a></span></li></ul></li><li><span><a href="#PID" data-toc-modified-id="PID-3.4">PID</a></span><ul class="toc-item"><li><span><a href="#dEdxTot-of-MIP" data-toc-modified-id="dEdxTot-of-MIP-3.4.1">dEdxTot of MIP</a></span></li><li><span><a href="#TPC-Gain-calibration" data-toc-modified-id="TPC-Gain-calibration-3.4.2">TPC Gain calibration</a></span></li><li><span><a href="#PID-Hypothesis-Profile" data-toc-modified-id="PID-Hypothesis-Profile-3.4.3">PID Hypothesis Profile</a></span></li></ul></li></ul></li></ul></div>

---
## Run Parameters

In [ ]:
def PlotRunParam(ParamName, ParamUnit, ParamArray, Runs):
  canvas = ROOT.TCanvas(ParamName+"Canvas", "Canvas", 1100, 400)
  canvas.SetBottomMargin(0.15)
  canvas.SetGridx()
  runListArrayint = np.array(Runs, dtype='int')
  NRuns = len(Runs)
  # Histogram of parameters
  hPar = ROOT.TH1F(ParamName, ParamName, NRuns, min(runListArrayint), max(runListArrayint))
  for i in range(len(ParamArray)):
      hPar.Fill(Runs[i], 
                    ParamArray[i])
      hPar.SetBinError(i+1,0)
  hPar.SetMarkerStyle(ROOT.kFullCircle)
  hPar.SetLineWidth(2)
  hPar.SetMarkerSize(1.5)
  hPar.SetStats(0)
  hPar.LabelsOption("v") # Vertical labels
  hPar.SetLineColor(ROOT.kBlack)
  hPar.GetXaxis().SetTitle("Run")
  hPar.GetYaxis().SetTitle(ParamName+" ({})".format(ParamUnit))
  hPar.Draw("SAME L P E")
  canvas.Update()
  # needs to return canvas also otherwise jupyter notebook doesn't display
  return hPar, canvas

In [ ]:
# Minor issue: these variables are not used if none of the QC.root files contain the RunParameters tree
# Table in HTML
RunParamTable = f"""
    <table style="border: 1px solid black; width: 50%;">
    <tr>
        <th style="background-color: #f2f2f2; text-align: center;">Run Number</th>
        <th style="background-color: #f2f2f2; text-align: center;">Interaction Rate (Hz)</th>
        <th style="background-color: #f2f2f2; text-align: center;">Duration (s)</th>
        <th style="background-color: #f2f2f2; text-align: center;">Magnetic Field (Amperes)</th>
    </tr>
    """
# Arrays for run parameters defined as initially containing only "?"
IntRatesAvg = np.full(len(runList), "?", dtype=object)
IntRatesStart = np.full(len(runList), "?", dtype=object)
IntRatesMid = np.full(len(runList), "?", dtype=object)
IntRatesEnd = np.full(len(runList), "?", dtype=object)
Durations = np.full(len(runList), "?", dtype=object)
BFields = np.full(len(runList), "?", dtype=object)
Polarity = np.full(len(runList), "?", dtype=object)
LegRunParam = runList.copy()
runsWithParamList=np.array([]) # List of runs with RunParameters tree
DisplayRunParam = False # Checks if at least 1 _QC.root has RunParameters
for i in range(len(runList)):
    RunParamTree = rootDataFile[i].Get("RunParameters")
    if RunParamTree:
        DisplayRunParam = True 
        runsWithParamList=np.append(runsWithParamList, (runList[i]))
        RunParamTree.GetEntry(0)
        IntRateAvg = getattr(RunParamTree, "IRavg")
        IntRatesAvg[i] = float(IntRateAvg)
        IntRateStart = getattr(RunParamTree, "IRstart")
        IntRatesStart[i] = float(IntRateStart)
        IntRateMid = getattr(RunParamTree, "IRmid")
        IntRatesMid[i] = float(IntRateMid)
        IntRateEnd = getattr(RunParamTree, "IRend")
        IntRatesEnd[i] = float(IntRateEnd)
        Duration = getattr(RunParamTree, "Duration")
        Durations[i] = float(Duration)
        BField = getattr(RunParamTree, "BField")
        BFields[i] = float(BField)
        Polarity[i] = "+" if BField > 0 else "-"
        LegIRavg = str(round(IntRatesAvg[i]/1000, 1))
        LegIRstart = str(round(IntRatesStart[i]/1000, 1))
        LegIRmid = str(round(IntRatesMid[i]/1000, 1))
        LegIRend = str(round(IntRatesEnd[i]/1000, 1))
        LegRunParam[i] += " (" + Polarity[i] + ") " + LegIRstart + " " + LegIRmid + " " + LegIRend + " (kHz)"           
        RunParamTable += f"""
            <tr style="text-align: center;">
                <td style="text-align: center;"><code>{runList[i]}</code></td>
                <td style="text-align: center;"><code>{IntRateAvg}</code></td>
                <td style="text-align: center;"><code>{Duration}</code></td>
                <td style="text-align: center;"><code>{BField}</code></td>
            </tr>
        """ # Row appended to table
    else: #if no RunParameters tree, fills table with "?"
        LegRunParam[i] += " (" + Polarity[i] + ") " + "?" + " kHz"          
        RunParamTable += f"""
            <tr style="text-align: center;">
                <td style="text-align: center;"><code>{runList[i]} (RunParameters tree not found)</code></td>
                <td style="text-align: center;"><code>?</code></td>
                <td style="text-align: center;"><code>?</code></td>
                <td style="text-align: center;"><code>?</code></td>
            </tr>
        """
if DisplayRunParam:
    display(HTML(RunParamTable))
    [hIR, canvasIR] = PlotRunParam("Interaction Rates", "Hz", IntRatesAvg[IntRatesAvg != "?"], runsWithParamList)
    canvasIR.Draw()
    [hDur, canvasDur] = PlotRunParam("Duration", "s", Durations[Durations  != "?"], runsWithParamList)
    canvasDur.Draw()
    [hBField, canvasBField] = PlotRunParam("BField", "Amperes", BFields[BFields != "?"], runsWithParamList)
    canvasBField.Draw()
else:
    display(HTML(f"No RunParameters trees found for the runs."))

---
## Quality Observer

### Clusters

In [ ]:
qualityDFClusters = pd.DataFrame({'runNumber':runList})
names = []
for i in range(72):
    if i < 18*2: roc = "IROC-"
    else: roc = "OROC-"
    if i%(18*2) < 18: side = "A"
    else : side = "C"
    names.append(roc+side+str(i%18).zfill(2))

In [ ]:
%%capture
objectName="c_ROCs_N_Clusters_1D;1"
trending = "mean"
error = ""
[trends,canvas] = drawMultiTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,pads=True,normalize=True,
                               log="none",yAxisRange=[0,500])
for iPad,pad in enumerate(canvas.GetListOfPrimitives()):
    trend = trends[iPad]
    [qualities,pad] = checkTrending(trend,canvas=pad,thresholds={"GOOD":1.5,"MEDIUM":3,"BAD":6})
    qualityDFClusters[names[iPad]] = qualities
    canvas.Update()
#canvas.Draw()

In [ ]:
if False:
    for i,row in qualityDFClusters.iterrows():
        probs = [columnName for (columnName, columnData) in row.items() if columnData in ["MEDIUM","BAD"]]
        if len(probs) != 0:
            print([row.runNumber]+probs)

In [ ]:
%jsroot on
myPalette = np.array([920, 414, 801, 633],dtype=np.int32)
ROOT.gStyle.SetPalette(4,myPalette)
ROOT.gStyle.SetGridStyle(1)
qualityLabels = {"NULL":0,"GOOD":1,"MEDIUM":2,"BAD":3}

canvas = ROOT.TCanvas("QualityMatrixClusters","",1200,600)
canvas.SetLeftMargin(0.15)
canvas.SetBottomMargin(0.15)
canvas.SetRightMargin(0.15)
canvas.SetGrid()

qualityHistClusters = ROOT.TH2I("QualityMatrixClusters","Clusters - Quality Matrix",
                        len(qualityDFClusters.index),min(qualityDFClusters.index),max(qualityDFClusters.index)+1,
                        len(qualityDFClusters.columns)-1,0,len(qualityDFClusters.columns)-1)
qualityHistClusters.SetCanExtend(ROOT.TH1.kAllAxes)
qualityHistClusters.SetStats(0)
for runIndex,run in enumerate(qualityDFClusters.runNumber):
    qualityHistClusters.GetXaxis().SetBinLabel(runIndex+1,str(run))
    qualityHistClusters.GetXaxis().SetTickLength( 0.03)
    for checkIndex,check in enumerate(qualityDFClusters.loc[:, qualityDFClusters.columns != "runNumber"].columns):
        #print(checkIndex,runIndex)
        qualityHistClusters.Fill(run,check,qualityLabels.get(qualityDFClusters.iloc[runIndex][check]))

qualityHistClusters.LabelsOption("u")
qualityHistClusters.Draw("COLZ")
qualityHistClusters.GetZaxis().SetRangeUser(-0.5,3.5);
qualityHistClusters.GetZaxis().SetTitle("Quality")
qualityHistClusters.LabelsOption("v")
canvas.Draw()

### Tracks

In [ ]:
%%capture
qualityDF = pd.DataFrame({'runNumber':runList})
objects = ["hPhiAside;1","hPhiCside;1","hEta;1","hPt;1","hSign;1","hdEdxTotMIP_TPC;1","hNClustersAfterCuts;1","hdEdxVsTgl;1","hdEdxTotMIPVsSnp_TPC;1"]
trending = "mean"
error = "meanError"
for objectName in objects:
    if objectName == "hdEdxTotMIP_TPC;1":
        trending = "fit(gaus,Sq,N,40,60)"
    else:
        trending = "mean"
    [trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                                   trend=trending,error=error,log="none",axis=1)
    [qualities,canvas] = checkTrending(trend,canvas=canvas,thresholds={"GOOD":1.5,"MEDIUM":3,"BAD":6})
    qualityDF[objectName] = qualities

In [ ]:
canvas = ROOT.TCanvas("QualityMatrixTracks","",1200,500)
canvas.SetLeftMargin(0.15)
canvas.SetBottomMargin(0.15)
canvas.SetRightMargin(0.15)
canvas.SetGrid()

qualityHist = ROOT.TH2I("QualityMatrixTracks","Tracks Observables - Quality Matrix",
                        len(qualityDF.index),min(qualityDF.index),max(qualityDF.index)+1,
                        len(qualityDF.columns)-1,0,len(qualityDF.columns)-1)
qualityHist.SetCanExtend(ROOT.TH1.kAllAxes)
qualityHist.SetStats(0)
for runIndex,run in enumerate(qualityDF.runNumber):
    qualityHist.GetXaxis().SetBinLabel(runIndex+1,str(run))
    qualityHist.GetXaxis().SetTickLength( 0.03)
    for checkIndex,check in enumerate(qualityDF.loc[:, qualityDF.columns != "runNumber"].columns):
        #print(checkIndex,runIndex)
        qualityHist.Fill(run,check,qualityLabels.get(qualityDF.iloc[runIndex][check]))

qualityHist.LabelsOption("u")
qualityHist.Draw("COLZ")
qualityHist.GetZaxis().SetRangeUser(-0.5,3.5)
qualityHist.GetZaxis().SetTitle("Quality")
qualityHist.LabelsOption("v")

# Update the canvas
canvas.Update()
canvas.Draw()

---
## Trendings
### Mean phi of tracks

In [ ]:
%jsroot on
objectName="hPhiAside;1"
trending = "mean"
error = "meanError"
leg = ROOT.TLegend()
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,axis=1,log="none")
[qualities,canvas] = checkTrending(trend,canvas=canvas,thresholds={"GOOD":1.5,"MEDIUM":3,"BAD":6})
canvas.SetCanvasSize(1000,400)
qualityDF[objectName] = qualities
canvas.Draw()

In [ ]:
%jsroot on
objectName="hPhiCside;1"
trending = "mean"
error = "meanError"
leg = ROOT.TLegend()
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,log="none")
[qualities,canvas] = checkTrending(trend,canvas=canvas,thresholds={"GOOD":1.5,"MEDIUM":3,"BAD":6})
qualityDF[objectName] = qualities
canvas.SetCanvasSize(1000,400)
#trend.SetTitle("Number of tracks - Trend")
canvas.Draw()

### Mean eta of tracks

In [ ]:
%jsroot on
objectName="hEta;1"
trending = "mean"
error = "meanError"
leg = ROOT.TLegend()
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,log="none")
[qualities,canvas] = checkTrending(trend,canvas=canvas,thresholds={"GOOD":1.5,"MEDIUM":3,"BAD":6})
qualityDF[objectName] = qualities
#trend.SetTitle("Number of tracks - Trend")
canvas.SetCanvasSize(1000,400)
canvas.Draw()

### Mean tracks transverse momentum of tracks

In [ ]:
%jsroot on
objectName="hPt;1"
trending = "mean"
error = "meanError"
leg = ROOT.TLegend()
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,log="none")
[qualities,canvas] = checkTrending(trend,canvas=canvas,thresholds={"GOOD":1.5,"MEDIUM":3,"BAD":6})
qualityDF[objectName] = qualities
#trend.SetTitle("Number of tracks - Trend")
canvas.SetCanvasSize(1000,400)
canvas.Draw()

### Mean charge sign of tracks

In [ ]:
%jsroot on
objectName="hSign;1"
trending = "mean"
error = "meanError"
leg = ROOT.TLegend()
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,log="none")
[qualities,canvas] = checkTrending(trend,canvas=canvas,thresholds={"GOOD":1.5,"MEDIUM":3,"BAD":6})
qualityDF[objectName] = qualities
#trend.SetTitle("Number of tracks - Trend")
canvas.SetCanvasSize(1000,400)
canvas.Draw()

### Mean dEdxTot of MIPs tracks

In [ ]:
%jsroot on
objectName="hdEdxTotMIP_TPC;1"
trending = "fit(gaus,Sq,N,40,60)"
error = "meanError"
leg = ROOT.TLegend()
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,log="none",yAxisRange=[30,70])
trend.Draw()
[qualities,canvas] = checkTrending(trend,canvas=canvas,thresholds={"GOOD":1.5,"MEDIUM":3,"BAD":6})

qualityDF[objectName] = qualities
#trend.GetYaxis().SetRangeUser(45,55)
#trend.SetTitle("Number of tracks - Trend")
canvas.SetCanvasSize(1000,400)
canvas.Draw()

**Mean number of clusters per tracks:**

In [ ]:
%jsroot on
objectName="hNClustersAfterCuts;1"
trending = "mean"
error = "meanError"
leg = ROOT.TLegend()
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,log="none")
[qualities,canvas] = checkTrending(trend,canvas=canvas,thresholds={"GOOD":1.5,"MEDIUM":3,"BAD":6})
qualityDF[objectName] = qualities
#trend.SetTitle("Number of tracks - Trend")
canvas.SetCanvasSize(1000,400)
canvas.Draw()

**Mean tan(lambda) of tracks:**

In [ ]:
%jsroot on
objectName="hdEdxVsTgl;1"
trending = "mean"
error = "meanError"
leg = ROOT.TLegend()
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,log="none")
[qualities,canvas] = checkTrending(trend,canvas=canvas,thresholds={"GOOD":1.5,"MEDIUM":3,"BAD":6})
qualityDF[objectName] = qualities
#trend.SetTitle("Number of tracks - Trend")
canvas.SetCanvasSize(1000,400)
canvas.Draw()

**Mean sin(phi) of tracks:**

In [ ]:
%jsroot on
objectName="hdEdxTotMIPVsSnp_TPC;1"
trending = "mean"
error = "meanError"
leg = ROOT.TLegend()
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,axis=1,log="none")
[qualities,canvas] = checkTrending(trend,canvas=canvas,thresholds={"GOOD":1.5,"MEDIUM":3,"BAD":6})
qualityDF[objectName] = qualities
#trend.SetTitle("Number of tracks - Trend")
canvas.SetCanvasSize(1000,400)
canvas.Draw()

**Mean DCA of A-side positive tracks:**

In [ ]:
%jsroot on
objectName="hDCAr_A_Pos;1"
trending = "mean"
error = "meanError"
leg = ROOT.TLegend()
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,axis=2,log="none")
[qualities,canvas] = checkTrending(trend,canvas=canvas,thresholds={"GOOD":1.5,"MEDIUM":3,"BAD":6})
qualityDF[objectName] = qualities
#trend.SetTitle("Number of tracks - Trend")
canvas.SetCanvasSize(1000,400)
canvas.Draw()

**Mean DCA of C-side positive tracks:**

In [ ]:
%jsroot on
objectName="hDCAr_C_Pos;1"
trending = "mean"
error = "meanError"
leg = ROOT.TLegend()
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,axis=2,log="none")
[qualities,canvas] = checkTrending(trend,canvas=canvas,thresholds={"GOOD":1.5,"MEDIUM":3,"BAD":6})
qualityDF[objectName] = qualities
#trend.SetTitle("Number of tracks - Trend")
canvas.SetCanvasSize(1000,400)
canvas.Draw()

**Mean DCA of A-side negative tracks:**

In [ ]:
%jsroot on
objectName="hDCAr_A_Neg;1"
trending = "mean"
error = "meanError"
leg = ROOT.TLegend()
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,axis=2,log="none")
[qualities,canvas] = checkTrending(trend,canvas=canvas,thresholds={"GOOD":1.5,"MEDIUM":3,"BAD":6})
qualityDF[objectName] = qualities
#trend.SetTitle("Number of tracks - Trend")
canvas.SetCanvasSize(1000,400)
canvas.Draw()

**Mean DCA of C-side negative tracks:**

In [ ]:
%jsroot on
objectName="hDCAr_C_Neg;1"
trending = "mean"
error = "meanError"
leg = ROOT.TLegend()
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,axis=2,log="none")
[qualities,canvas] = checkTrending(trend,canvas=canvas,thresholds={"GOOD":1.5,"MEDIUM":3,"BAD":6})
qualityDF[objectName] = qualities
#trend.SetTitle("Number of tracks - Trend")
canvas.SetCanvasSize(1000,400)
canvas.Draw()

---
## Plots

In [ ]:
ROOT.gStyle.SetPalette(57)
ROOT.gStyle.SetGridStyle(3)
ROOT.gStyle.SetGridWidth(1)

### Clusters
**Number of clusters per pad, A-side:**

In [ ]:
%jsroot on
objectName="h2DRPhinClustersAside"
if checkIfExists(rootDataFile,objectName):
    [quants,legend,canvas] = drawProjection(objectName,rootDataFile,size=[1200,400])
    updateColors(quants,cols)
    canvas.SetGridy(1)
    for quant in quants:
        quant.Scale(1/quant.GetEntries())
        quant.SetTitle(objectName+" Projection")
    updateRanges(quants)
    canvas.cd()
    sectorBoundary = ROOT.TLine(0,quant.GetMinimum(),0,quant.GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,quant.GetMinimum(),(j*2*math.pi)/18,quant.GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,quant.GetMaximum()*0.95+quant.GetMinimum()*0.05,str(j))
    
    histoOut, canvasOut = outlierDetection(objectName,quants,runList)

    legend.Draw()
    canvas.Draw()

In [ ]:
%jsroot on
objectName="h2DRPhinClustersAside"
if checkIfExists(rootDataFile,objectName):
    [quants,legend,canvas] = drawProjection(objectName,rootDataFile, axis="y", size=[1200,400])
    updateColors(quants,cols)
    canvas.SetGridy(1)
    for quant in quants:
        quant.Scale(1/quant.GetEntries())
        quant.SetTitle(objectName+" Projection")
    updateRanges(quants)
    canvas.cd()
    sectorBoundary = ROOT.TLine(0,quant.GetMinimum(),0,quant.GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,quant.GetMinimum(),(j*2*math.pi)/18,quant.GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,quant.GetMaximum()*0.95+quant.GetMinimum()*0.05,str(j))
    
    histoOut, canvasOut = outlierDetection(objectName,quants,runList)
    
    legend.Draw()
    canvas.Draw()

**Number of clusters per pad, C-side:**

In [ ]:
%jsroot on
objectName="h2DRPhinClustersCside"
if checkIfExists(rootDataFile,objectName):
    [quants,legend,canvas] = drawProjection(objectName,rootDataFile,size=[1200,400])
    updateColors(quants,cols)
    canvas.SetGridy(1)
    for quant in quants:
        quant.Scale(1/quant.GetEntries())
        quant.SetTitle(objectName+" Projection")
    updateRanges(quants)
    canvas.cd()
    sectorBoundary = ROOT.TLine(0,quant.GetMinimum(),0,quant.GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,quant.GetMinimum(),(j*2*math.pi)/18,quant.GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,quant.GetMaximum()*0.95+quant.GetMinimum()*0.05,str(j))
    
    histoOut, canvasOut = outlierDetection(objectName,quants,runList)
    
    legend.Draw()
    canvas.Draw()

In [ ]:
%jsroot on
objectName="h2DRPhinClustersCside"
if checkIfExists(rootDataFile,objectName):
    [quants,legend,canvas] = drawProjection(objectName,rootDataFile, axis="y", size=[1200,400])
    updateColors(quants,cols)
    canvas.SetGridy(1)
    for quant in quants:
        quant.Scale(1/quant.GetEntries())
        quant.SetTitle(objectName+" Projection")
    updateRanges(quants)
    canvas.cd()
    sectorBoundary = ROOT.TLine(0,quant.GetMinimum(),0,quant.GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,quant.GetMinimum(),(j*2*math.pi)/18,quant.GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,quant.GetMaximum()*0.95+quant.GetMinimum()*0.05,str(j))
    
    histoOut, canvasOut = outlierDetection(objectName,quants,runList)
    
    legend.Draw()
    canvas.Draw()

---
**Cluster total charge per pad, A-side:**

In [ ]:
%jsroot on
objectName="h2DRPhiqTotAside"
if checkIfExists(rootDataFile,objectName):
    [quants,legend,canvas] = drawProjection(objectName,rootDataFile,size=[1200,400])
    updateColors(quants,cols)
    canvas.SetGridy(1)
    for quant in quants:
        quant.Scale(1/quant.GetEntries())
        quant.SetTitle(objectName+" Projection")
    updateRanges(quants)
    canvas.cd()
    sectorBoundary = ROOT.TLine(0,quant.GetMinimum(),0,quant.GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,quant.GetMinimum(),(j*2*math.pi)/18,quant.GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,quant.GetMaximum()*0.95+quant.GetMinimum()*0.05,str(j))
    
    histoOut, canvasOut = outlierDetection(objectName,quants,runList)
    
    legend.Draw()
    canvas.Draw()

In [ ]:
%jsroot on
objectName="h2DRPhiqTotAside"
if checkIfExists(rootDataFile,objectName):
    [quants,legend,canvas] = drawProjection(objectName,rootDataFile, axis="y", size=[1200,400])
    updateColors(quants,cols)
    canvas.SetGridy(1)
    for quant in quants:
        quant.Scale(1/quant.GetEntries())
        quant.SetTitle(objectName+" Projection")
    updateRanges(quants)
    canvas.cd()
    sectorBoundary = ROOT.TLine(0,quant.GetMinimum(),0,quant.GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,quant.GetMinimum(),(j*2*math.pi)/18,quant.GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,quant.GetMaximum()*0.95+quant.GetMinimum()*0.05,str(j))
    
    histoOut, canvasOut = outlierDetection(objectName,quants,runList)
    
    legend.Draw()
    canvas.Draw()

**Cluster total charge per pad, C-side:**

In [ ]:
%jsroot on
objectName="h2DRPhiqTotCside"
if checkIfExists(rootDataFile,objectName):
    [quants,legend,canvas] = drawProjection(objectName,rootDataFile,size=[1200,400])
    updateColors(quants,cols)
    canvas.SetGridy(1)
    for quant in quants:
        quant.Scale(1/quant.GetEntries())
        quant.SetTitle(objectName+" Projection")
    updateRanges(quants)
    canvas.cd()
    sectorBoundary = ROOT.TLine(0,quant.GetMinimum(),0,quant.GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,quant.GetMinimum(),(j*2*math.pi)/18,quant.GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,quant.GetMaximum()*0.95+quant.GetMinimum()*0.05,str(j))
    
    histoOut, canvasOut = outlierDetection(objectName,quants,runList)
    
    legend.Draw()
    canvas.Draw()

In [ ]:
%jsroot on
objectName="h2DRPhiqTotCside"
if checkIfExists(rootDataFile,objectName):
    [quants,legend,canvas] = drawProjection(objectName,rootDataFile, axis="y", size=[1200,400])
    updateColors(quants,cols)
    canvas.SetGridy(1)
    for quant in quants:
        quant.Scale(1/quant.GetEntries())
        quant.SetTitle(objectName+" Projection")
    updateRanges(quants)
    canvas.cd()
    sectorBoundary = ROOT.TLine(0,quant.GetMinimum(),0,quant.GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,quant.GetMinimum(),(j*2*math.pi)/18,quant.GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,quant.GetMaximum()*0.95+quant.GetMinimum()*0.05,str(j))
    
    histoOut, canvasOut = outlierDetection(objectName,quants,runList)
    
    legend.Draw()
    canvas.Draw()

---
### Tracks
#### Geometrical distributions of tracks
**Phi distribution A-side tracks:**

In [ ]:
%jsroot on
objectName="hPhiAside"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=True,legend=True,legendNames=LegRunParam,pads=False,
                                               drawOption="HIST E SAME",yAxisRange=[2.2e-3,3.2e-3],grid=False,size=[1200,400])
    #Sector boundatries
    pad1.cd()
    sectorBoundary = ROOT.TLine(0,hist[0].GetMinimum(),0,hist[0].GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,hist[0].GetMinimum(),(j*2*math.pi)/18,hist[0].GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,hist[0].GetMaximum(),str(j))
    
    histoOut, canvasOut = outlierDetection(objectName,hist,runList)

    pad1.SetGridy(1)
    canvas.SetCanvasSize(1000,400)
    updateColors(hist,cols)
    legend.SetHeader("Normalized to integral, #Run polarity IRstart IRmid IRend", "C")
    legend.Draw()
    canvas.Draw()

**Phi distribution C-side tracks:**

In [ ]:
%jsroot on
objectName="hPhiCside"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=True,legend=True,legendNames=LegRunParam,pads=False,
                                               drawOption="HIST E SAME",yAxisRange=[2.2e-3,3.2e-3],grid=False,size=[1200,400])
    updateColors(hist,cols)
    #Sector boundatries
    pad1.cd()
    sectorBoundary = ROOT.TLine(0,hist[0].GetMinimum(),0,hist[0].GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,hist[0].GetMinimum(),(j*2*math.pi)/18,hist[0].GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,hist[0].GetMaximum(),str(j))
    
    histoOut, canvasOut = outlierDetection(objectName,hist,runList)

    pad1.SetGridy(1)
    canvas.SetCanvasSize(1000,400)
    legend.SetHeader("Normalized to integral, #Run polarity IRstart IRmid IRend", "C")
    legend.Draw()
    canvas.Draw()

**Eta distribution:**

In [ ]:
%jsroot on
objectName="hEta"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=True,legend=True,legendNames=LegRunParam,pads=False,
                                               drawOption="HIST E SAME",xAxisRange=[-1.1,1.1],grid=True,size=[1000,400])
    updateColors(hist,cols)
    updateRanges(hist)

    histoOut, canvasOut = outlierDetection(objectName,hist,runList)

    pad1.SetGrid(1)
    canvas.SetCanvasSize(700,400)
    legend.SetHeader("Normalized to integral, #Run polarity IRstart IRmid IRend", "C")
    legend.Draw()
    canvas.Draw()

In [ ]:
%jsroot on
objectName="h2DEtaPhi"
if checkIfExists(rootDataFile,objectName):
    [quants,legend,canvas] = drawQuantileProjection(objectName,rootDataFile,size=[1200,400])
    updateColors(quants,cols)
    canvas.SetGridy(1)
    canvas.SetCanvasSize(1000,400)
    for quant in quants:
        quant.GetYaxis().SetRangeUser(-.1,.1)
        quant.SetTitle("Median eta vs phi")
    canvas.cd()
    sectorBoundary = ROOT.TLine(0,quant.GetMinimum(),0,quant.GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,quant.GetMinimum(),(j*2*math.pi)/18,quant.GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,quant.GetMaximum()*0.95+quant.GetMinimum()*0.05,str(j))
    
    histoOut, canvasOut = outlierDetection(objectName,quants,runList)

    legend.Draw()
    canvas.Draw()

#### Track properties
**Transverse momentum**

In [ ]:
%jsroot on
objectName="hPt"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=True,legend=True,legendNames=LegRunParam,pads=False,
                                               drawOption="HIST E SAME",log="logxy",
                                               grid=True,size=[1000,400])
    
    histoOut, canvasOut = outlierDetection(objectName,hist,runList)

    updateColors(hist,cols)
    pad1.SetGrid(1)
    canvas.SetCanvasSize(700,400)
    legend.SetHeader("Normalized to integral, #Run polarity IRstart IRmid IRend", "C")
    legend.Draw()
    canvas.Draw()

**Track sign**

In [ ]:
%jsroot on
objectName="hSign"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=True,legend=True,legendNames=LegRunParam,pads=False,
                                               drawOption="SAME HIST",grid=True,size=[1000,400])

    histoOut, canvasOut = outlierDetection(objectName,hist,runList)
    
    updateColors(hist,cols)
    pad1.SetGrid(1)
    canvas.SetCanvasSize(700,400)
    legend.SetHeader("Normalized to integral, #Run polarity IRstart IRmid IRend", "C")
    legend.Draw()
    canvas.Draw()

**Charge / transverse momentum**

In [ ]:
%jsroot on
objectName="hQOverPt"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=True,legend=True,legendNames=LegRunParam,pads=False,
                                                        drawOption="HIST E SAME",grid=True,size=[1000,400])
    
    histoOut, canvasOut = outlierDetection(objectName,hist,runList)
    
    updateColors(hist,cols)
    pad1.SetGrid(1)
    canvas.SetCanvasSize(700,400)
    legend.SetHeader("Normalized to integral, #Run polarity IRstart IRmid IRend", "C")
    legend.Draw()
    canvas.Draw()

**Number of clusters per track**

In [ ]:
%jsroot on
objectNames=["hNClustersAfterCuts","crossedRows","sharedClusters"]
if all([checkIfExists(rootDataFile,objectName) for objectName in objectNames]):
    bigCanvas = ROOT.TCanvas("","",1200,400)
    bigCanvas.Divide(3,1)
    [hist1,legend1,canvas1,pad1] = drawHistograms(objectNames[0],rootDataFile,xAxisRange = [50,160],normalize=True,legend=True,legendNames=LegRunParam)
    [hist2,legend2,canvas2,pad2] = drawHistograms(objectNames[1],rootDataFile,xAxisRange = [50,160],normalize=True,legend=True,legendNames=LegRunParam)
    [hist3,legend3,canvas3,pad3] = drawHistograms(objectNames[2],rootDataFile,xAxisRange = [0,152],normalize=True,legend=True,legendNames=LegRunParam)
    #canvas.SetCanvasSize(700,400)
    #hist.SetGrid(1)
    updateColors(hist1,cols)
    updateColors(hist2,cols)
    updateColors(hist3,cols)
    updateRanges(hist1)
    updateRanges(hist2)
    updateRanges(hist3)
    histoOut, canvasOut = outlierDetection(objectName,hist1,runList)
    bigCanvas.cd(1)
    ROOT.gPad.SetGrid(1)
    for hist in hist1 : hist.Draw("HIST E SAME")
    legend1.SetHeader("Normalized to integral, #Run polarity IRstart IRmid IRend", "C")
    legend1.Draw()
    histoOut, canvasOut = outlierDetection(objectName,hist2,runList)
    bigCanvas.cd(2)
    ROOT.gPad.SetGrid(1)
    for hist in hist2 : hist.Draw("HIST E SAME")
    legend2.SetHeader("Normalized to integral, #Run polarity IRstart IRmid IRend", "C")
    legend2.Draw()
    histoOut, canvasOut = outlierDetection(objectName,hist3,runList)
    bigCanvas.cd(3)
    ROOT.gPad.SetGridx(1)
    ROOT.gPad.SetLogy()
    for hist in hist3 : hist.Draw("HIST E SAME")
    legend3.Draw()
    bigCanvas.Draw()

**nClusters vs phi**

In [ ]:
%jsroot on
objectName="h2DNClustersPhiAside"
if checkIfExists(rootDataFile,objectName):
    [quants,legend,canvas] = drawQuantileProjection(objectName,rootDataFile,size=[1200,400])
    updateColors(quants,cols)
    canvas.SetGridy(1)
    canvas.SetCanvasSize(1000,400)
    for quant in quants:
        quant.GetYaxis().SetRangeUser(60,160)
        quant.SetTitle("A-Side - nClusters per track vs phi")
    canvas.cd()
    sectorBoundary = ROOT.TLine(0,quant.GetMinimum(),0,quant.GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,quant.GetMinimum(),(j*2*math.pi)/18,quant.GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,quant.GetMaximum()*0.95+quant.GetMinimum()*0.05,str(j))
    
    histoOut, canvasOut = outlierDetection(objectName,quants,runList)

    legend.Draw()
canvas.Draw()

In [ ]:
%jsroot on
objectName="h2DNClustersPhiCside"
if checkIfExists(rootDataFile,objectName):
    [quants,legend,canvas] = drawQuantileProjection(objectName,rootDataFile,size=[1200,400])
    updateColors(quants,cols)
    canvas.SetGridy(1)
    canvas.SetCanvasSize(1000,400)
    for quant in quants:
        quant.GetYaxis().SetRangeUser(60,160)
        quant.SetTitle("C-Side - nClusters per track vs phi")
    canvas.cd()
    sectorBoundary = ROOT.TLine(0,quant.GetMinimum(),0,quant.GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,quant.GetMinimum(),(j*2*math.pi)/18,quant.GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,quant.GetMaximum()*0.95+quant.GetMinimum()*0.05,str(j))
    
    histoOut, canvasOut = outlierDetection(objectName,quants,runList)
    
    legend.Draw()
canvas.Draw()

**nClusters vs eta**

In [ ]:
%jsroot on
objectName="h2DNClustersEta"
if checkIfExists(rootDataFile,objectName):
    [quants,legend,canvas] = drawQuantileProjection(objectName,rootDataFile,size=[1200,400])
    updateColors(quants,cols)
    canvas.SetGrid(1)
    canvas.SetCanvasSize(700,400)
    for quant in quants:
        quant.GetXaxis().SetRangeUser(-1.1,1.1)
        quant.GetYaxis().SetRangeUser(60,160)
    legend.Draw()

    histoOut, canvasOut = outlierDetection(objectName,quants,runList)
    
canvas.Draw()

**nClusters vs transverse momentum**

In [ ]:
%jsroot on
objectName="h2DNClustersPt"
if checkIfExists(rootDataFile,objectName):
    [quants,legend,canvas] = drawQuantileProjection(objectName,rootDataFile,size=[1000,400])
    updateColors(quants,cols)
    canvas.SetGrid(1)
    canvas.SetCanvasSize(700,400)
    canvas.SetLogx()
    for quant in quants:
        quant.GetYaxis().SetRangeUser(60,160)
    legend.Draw()

    histoOut, canvasOut = outlierDetection(objectName,quants,runList)
    
canvas.Draw()

### DCAr

#### DCAr vs phi

In [ ]:
%jsroot on
objectName="hDCAr_A_Pos"
if checkIfExists(rootDataFile,objectName):
    [quants,legend,canvas] = drawQuantileProjection(objectName,rootDataFile,size=[1200,400])
    updateColors(quants,cols)
    canvas.SetGridy(1)
    canvas.SetCanvasSize(1000,350)
    for quant in quants:
        quant.GetYaxis().SetRangeUser(-3,3)
        quant.SetTitle("DCAr A-Side - Positive")
    canvas.cd()
    sectorBoundary = ROOT.TLine(0,quant.GetMinimum(),0,quant.GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,quant.GetMinimum(),(j*2*math.pi)/18,quant.GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,quant.GetMaximum()*0.95+quant.GetMinimum()*0.05,str(j))
    
    histoOut, canvasOut = outlierDetection(objectName,quants,runList)

    legend.Draw()
    canvas.Draw()

In [ ]:
%jsroot on
objectName="hDCAr_A_Neg"
if checkIfExists(rootDataFile,objectName):
    [quants,legend,canvas] = drawQuantileProjection(objectName,rootDataFile,size=[1200,400])
    updateColors(quants,cols)
    canvas.SetGridy(1)
    canvas.SetCanvasSize(1000,350)
    for quant in quants:
        quant.GetYaxis().SetRangeUser(-3,3)
        quant.SetTitle("DCAr A-Side - Negative")
    canvas.cd()
    sectorBoundary = ROOT.TLine(0,quant.GetMinimum(),0,quant.GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,quant.GetMinimum(),(j*2*math.pi)/18,quant.GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,quant.GetMaximum()*0.95+quant.GetMinimum()*0.05,str(j))
    
    histoOut, canvasOut = outlierDetection(objectName,quants,runList)
    
    legend.Draw()
    canvas.Draw()

In [ ]:
%jsroot on
objectName="hDCAr_C_Pos"
if checkIfExists(rootDataFile,objectName):
    [quants,legend,canvas] = drawQuantileProjection(objectName,rootDataFile,size=[1200,400])
    updateColors(quants,cols)
    canvas.SetGridy(1)
    canvas.SetCanvasSize(1000,350)
    for quant in quants:
        quant.GetYaxis().SetRangeUser(-3,3)
        quant.SetTitle("DCAr C-Side - Positive")
    canvas.cd()
    sectorBoundary = ROOT.TLine(0,quant.GetMinimum(),0,quant.GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,quant.GetMinimum(),(j*2*math.pi)/18,quant.GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,quant.GetMaximum()*0.95+quant.GetMinimum()*0.05,str(j))

    histoOut, canvasOut = outlierDetection(objectName,quants,runList)
    
    legend.Draw()
    canvas.Draw()

In [ ]:
%jsroot on
objectName="hDCAr_C_Neg"
if checkIfExists(rootDataFile,objectName):
    [quants,legend,canvas] = drawQuantileProjection(objectName,rootDataFile,size=[1200,400])
    updateColors(quants,cols)
    canvas.SetGridy(1)
    canvas.SetCanvasSize(1000,350)
    for quant in quants:
        quant.GetYaxis().SetRangeUser(-3,3)
        quant.SetTitle("DCAr C-Side - Negative")
    canvas.cd()
    sectorBoundary = ROOT.TLine(0,quant.GetMinimum(),0,quant.GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,quant.GetMinimum(),(j*2*math.pi)/18,quant.GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,quant.GetMaximum()*0.95+quant.GetMinimum()*0.05,str(j))

    histoOut, canvasOut = outlierDetection(objectName,quants,runList)
    
    legend.Draw()
    canvas.Draw()

#### DCAr vs eta

In [ ]:
%jsroot on
objectName="hDCArVsEtaPos"
title = "DCAr vs eta - Positive"
if checkIfExists(rootDataFile,objectName):
    [quants,legend,canvas] = drawQuantileProjection(objectName,rootDataFile,size=[1200,400])
    updateColors(quants,cols)
    canvas.SetGrid(1)
    canvas.SetCanvasSize(1000,350)
    for quant in quants:
        quant.GetYaxis().SetRangeUser(-3,3)
        quant.GetXaxis().SetRangeUser(-1,1)
        quant.SetTitle(title)
    
    histoOut, canvasOut = outlierDetection(objectName,quants,runList)
    
    canvas.cd()
    legend.Draw()
    canvas.Draw()

In [ ]:
%jsroot on
objectName="hDCArVsEtaNeg"
title = "DCAr vs eta - Negative"
if checkIfExists(rootDataFile,objectName):
    [quants,legend,canvas] = drawQuantileProjection(objectName,rootDataFile,size=[1200,400])
    updateColors(quants,cols)
    canvas.SetGrid(1)
    canvas.SetCanvasSize(1000,350)
    for quant in quants:
        quant.GetYaxis().SetRangeUser(-3,3)
        quant.GetXaxis().SetRangeUser(-1,1)
        quant.SetTitle(title)
    
    histoOut, canvasOut = outlierDetection(objectName,quants,runList)
    
    canvas.cd()
    legend.Draw()
    canvas.Draw()

#### DCAr vs pT

In [ ]:
%jsroot on
objectName="hDCArVsPtPos"
title = "DCAr vs pT - Positive"
if checkIfExists(rootDataFile,objectName):
    [quants,legend,canvas] = drawQuantileProjection(objectName,rootDataFile,size=[1200,400])
    updateColors(quants,cols)
    canvas.SetGrid(1)
    canvas.SetCanvasSize(1000,350)
    for quant in quants:
        quant.GetYaxis().SetRangeUser(-3,3)
        quant.GetXaxis().SetRangeUser(0.1,20)
        quant.SetTitle(title)
    
    histoOut, canvasOut = outlierDetection(objectName,quants,runList)
    
    canvas.cd()
    legend.Draw()
    canvas.SetLogx()
    canvas.Draw()

In [ ]:
%jsroot on
objectName="hDCArVsPtNeg"
title = "DCAr vs pT - Negative"
if checkIfExists(rootDataFile,objectName):
    [quants,legend,canvas] = drawQuantileProjection(objectName,rootDataFile,size=[1200,400])
    updateColors(quants,cols)
    canvas.SetGrid(1)
    canvas.SetCanvasSize(1000,350)
    for quant in quants:
        quant.GetYaxis().SetRangeUser(-3,3)
        quant.GetXaxis().SetRangeUser(0.1,20)
        quant.SetTitle(title)
    
    histoOut, canvasOut = outlierDetection(objectName,quants,runList)
    
    canvas.cd()
    legend.Draw()
    canvas.SetLogx()
    canvas.Draw()

#### DCAr vs nClusters

In [ ]:
%jsroot on
objectName="hDCArVsNClsPos"
title = "DCAr vs nClusters - Positive"
if checkIfExists(rootDataFile,objectName):
    [quants,legend,canvas] = drawQuantileProjection(objectName,rootDataFile,size=[1200,400])
    updateColors(quants,cols)
    canvas.SetGrid(1)
    canvas.SetCanvasSize(1000,350)
    for quant in quants:
        quant.GetYaxis().SetRangeUser(-3,3)
        quant.GetXaxis().SetRangeUser(60,160)
        quant.SetTitle(title)
    
    histoOut, canvasOut = outlierDetection(objectName,quants,runList)
    
    canvas.cd()
    legend.Draw()
    canvas.Draw()

In [ ]:
%jsroot on
objectName="hDCArVsNClsNeg"
title = "DCAr vs nClusters - Negative"
if checkIfExists(rootDataFile,objectName):
    [quants,legend,canvas] = drawQuantileProjection(objectName,rootDataFile,size=[1200,400])
    updateColors(quants,cols)
    canvas.SetGrid(1)
    canvas.SetCanvasSize(1000,350)
    for quant in quants:
        quant.GetYaxis().SetRangeUser(-3,3)
        quant.GetXaxis().SetRangeUser(60,160)
        quant.SetTitle(title)
    
    histoOut, canvasOut = outlierDetection(objectName,quants,runList)
    
    canvas.cd()
    legend.Draw()
    canvas.Draw()

### PID
#### dEdxTot of MIP

In [ ]:
%jsroot on
objects = ["hdEdxTotMIP_" + region for region in ["TPC","IROC","OROC1","OROC2","OROC3"]]
if all([checkIfExists(rootDataFile,objectName) for objectName in objects]):
    hists = []
    for objectName in objects:
        [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=True,legend=True,legendNames=LegRunParam,pads=False,
                                               drawOption="HIST E SAME",log="logx")
        updateColors(hist,cols)
        hists.append(copy(hist))
    c = ROOT.TCanvas("MIP","MIP",1100,600)
    c.Divide(2)
    leftPad = c.cd(1)
    leftPad.SetGrid(1)
    ranges = [updateRanges(histograms) for histograms in hists]

    histoOut, canvasOut = outlierDetection(objectName,hists[0],runList)

    for hist in hists[0]:
        hist.Draw("HIST E SAME")
        hist.SetStats(0)
    #Draw target 50
    targets = [ROOT.TLine(50,yRange[0],50,yRange[1]) for yRange in ranges]
    for i in range(len(objects)):
        targets[i].SetLineStyle(2)
        targets[i].SetLineColor(3)
        targets[i].SetLineWidth(3)
    targets[0].Draw()
    legend.SetHeader("Normalized to integral, #Run polarity IRstart IRmid IRend", "C")
    legend.Draw()
    rightPad = c.cd(2)
    rightPad.Divide(2,2)
    for i in range(1,5):
        rightPad.cd(i)
        ROOT.gPad.SetGrid(1)

        histoOut, canvasOut = outlierDetection(objectName,hists[i],runList)
        
        for hist in hists[i]:
            hist.Draw("HIST E SAME")
            hist.SetStats(0)
        targets[i].Draw()
    c.Draw()

#### TPC Gain calibration
**dEdx of MIP vs sector**

In [ ]:
%jsroot on
objectName="hdEdxTotMIPVsSec_TPC"

if checkIfExists(rootDataFile,objectName):
    [fits_mean,legend_mean,canvas_mean] = sliceAndFit(objectName,rootDataFile,size=[1200,400])
    updateColors(fits_mean,cols)
    canvas_mean.SetGridy(1)
    canvas_mean.SetCanvasSize(1000,350)
    for fit in fits_mean:
        fit.GetYaxis().SetRangeUser(45,55)
        fit.SetTitle("TPC Mean MIP Q_{Tot} vs Sector")
    sectorBoundary = ROOT.TLine()
    sectorBoundary.SetLineStyle(2)
    sectorBoundary.SetLineColor(ROOT.kBlack)
    sectorBoundary.SetLineWidth(2)
    sectorBoundary.DrawLine(0,50,36,50)

    histoOut, canvasOut = outlierDetection(objectName,fits_mean,runList)

    canvas_mean.cd()
    canvas_mean.Draw()
    
if checkIfExists(rootDataFile,objectName):
    [fits_width,legend_width,canvas_width] = sliceAndFitWidth(objectName,rootDataFile,size=[1200,400])
    updateColors(fits_width,cols)
    canvas_width.SetGridy(1)
    canvas_width.SetCanvasSize(1000,350)
    for fit in fits_width:
        fit.GetYaxis().SetRangeUser(0,10)
        fit.SetTitle("TPC Width MIP Q_{Tot} vs Sector")
    sectorBoundary = ROOT.TLine()
    sectorBoundary.SetLineStyle(2)
    sectorBoundary.SetLineColor(ROOT.kBlack)
    sectorBoundary.SetLineWidth(2)
    sectorBoundary.DrawLine(0,50,36,50)

    histoOut, canvasOut = outlierDetection(objectName,fits_width,runList)

    canvas_width.cd()
    canvas_width.Draw()


**dEdx of Mip vs nClusters**

In [ ]:
%jsroot on
objectName="hdEdxTotMIPVsNcl_TPC"

if checkIfExists(rootDataFile,objectName):
    [fits_mean,legend_mean,canvas_mean] = sliceAndFit(objectName,rootDataFile,size=[1200,400])
    updateColors(fits_mean,cols)
    canvas_mean.SetGridy(1)
    canvas_mean.SetCanvasSize(1000,350)
    for fit in fits_mean:
        fit.GetYaxis().SetRangeUser(45,55)
        fit.GetXaxis().SetRangeUser(60,160)
        fit.SetTitle("TPC Mean MIP Q_{Tot} vs nClusters")
    sectorBoundary = ROOT.TLine()
    sectorBoundary.SetLineStyle(2)
    sectorBoundary.SetLineColor(ROOT.kBlack)
    sectorBoundary.SetLineWidth(2)
    sectorBoundary.DrawLine(60,50,153,50)

    histoOut, canvasOut = outlierDetection(objectName,fits_mean,runList)

    canvas_mean.cd()
    canvas_mean.Draw()

if checkIfExists(rootDataFile,objectName):
    [fits_width,legend_width,canvas_width] = sliceAndFitWidth(objectName,rootDataFile,size=[1200,400])
    updateColors(fits_width,cols)
    canvas_width.SetGridy(1)
    canvas_width.SetCanvasSize(1000,350)
    for fit in fits_width:
        fit.GetYaxis().SetRangeUser(0,10)
        fit.SetTitle("TPC Width MIP Q_{Tot} vs nClusters")
    sectorBoundary = ROOT.TLine()
    sectorBoundary.SetLineStyle(2)
    sectorBoundary.SetLineColor(ROOT.kBlack)
    sectorBoundary.SetLineWidth(2)
    sectorBoundary.DrawLine(0,50,36,50)

    histoOut, canvasOut = outlierDetection(objectName,fits_width,runList)

    canvas_width.cd()
    canvas_width.Draw()


**dEdx of Mip vs sin(phi)**

In [ ]:
%jsroot on
objectName="hdEdxTotMIPVsSnp_TPC"

if checkIfExists(rootDataFile,objectName):
    [fits_mean,legend_mean,canvas_mean] = sliceAndFit(objectName,rootDataFile,fitRange=[40,60],size=[1200,400])
    updateColors(fits_mean,cols)
    canvas_mean.SetGridy(1)
    canvas_mean.SetCanvasSize(1000,350)
    for fit in fits_mean:
        fit.GetYaxis().SetRangeUser(35,65)
        fit.GetXaxis().SetRangeUser(-1,1)
        fit.SetTitle("TPC Mean MIP Q_{Tot} vs sin(phi)")
        sectorBoundary = ROOT.TLine()
        sectorBoundary.SetLineStyle(2)
        sectorBoundary.SetLineColor(ROOT.kBlack)
        sectorBoundary.SetLineWidth(2)
        sectorBoundary.DrawLine(-1,50,1,50)
    
    histoOut, canvasOut = outlierDetection(objectName,fits_mean,runList)

    canvas_mean.cd()
    canvas_mean.Draw()

if checkIfExists(rootDataFile,objectName):
    [fits_width,legend_width,canvas_width] = sliceAndFitWidth(objectName,rootDataFile,size=[1200,400])
    updateColors(fits_width,cols)
    canvas_width.SetGridy(1)
    canvas_width.SetCanvasSize(1000,350)
    for fit in fits_width:
        fit.GetYaxis().SetRangeUser(0,30)
        fit.SetTitle("TPC Width MIP Q_{Tot} vs sin(phi)")
    sectorBoundary = ROOT.TLine()
    sectorBoundary.SetLineStyle(2)
    sectorBoundary.SetLineColor(ROOT.kBlack)
    sectorBoundary.SetLineWidth(2)
    sectorBoundary.DrawLine(0,50,36,50)

    histoOut, canvasOut = outlierDetection(objectName,fits_width,runList)

    canvas_width.cd()
    canvas_width.Draw()


**dEdx of Mip vs tan(lambda)**

In [ ]:
%jsroot on
objectName="hdEdxTotMIPVsTgl_TPC"
if checkIfExists(rootDataFile,objectName):
    [fits_mean,legend_mean,canvas_mean] = sliceAndFit(objectName,rootDataFile,fitRange=[40,60],size=[1200,400])
    updateColors(fits_mean,cols)
    canvas_mean.SetGridy(1)
    canvas_mean.SetCanvasSize(1000,350)
    for fit in fits_mean:
        fit.GetYaxis().SetRangeUser(45,55)
        fit.GetXaxis().SetRangeUser(-2,2)
        fit.SetTitle("TPC Mean MIP Q_{Tot} vs tan(lambda)")
        sectorBoundary = ROOT.TLine()
        sectorBoundary.SetLineStyle(2)
        sectorBoundary.SetLineColor(ROOT.kBlack)
        sectorBoundary.SetLineWidth(2)
        sectorBoundary.DrawLine(-2,50,2,50)

    histoOut, canvasOut = outlierDetection(objectName,fits_mean,runList)
    
    canvas_mean.cd()
    canvas_mean.Draw()
    
if checkIfExists(rootDataFile,objectName):
    [fits_width,legend_width,canvas_width] = sliceAndFitWidth(objectName,rootDataFile,size=[1200,400])
    updateColors(fits_width,cols)
    canvas_width.SetGridy(1)
    canvas_width.SetCanvasSize(1000,350)
    for fit in fits_width:
        fit.GetYaxis().SetRangeUser(0,10)
        fit.SetTitle("TPC Width MIP Q_{Tot} vs tan(lambda)")
    sectorBoundary = ROOT.TLine()
    sectorBoundary.SetLineStyle(2)
    sectorBoundary.SetLineColor(ROOT.kBlack)
    sectorBoundary.SetLineWidth(2)
    sectorBoundary.DrawLine(0,50,36,50)

    histoOut, canvasOut = outlierDetection(objectName,fits_width,runList)

    canvas_width.cd()
    canvas_width.Draw()


#### PID Hypothesis Profile

In [ ]:
%jsroot on
if checkIfExists(rootDataFile, "CdEdxPIDHypothesisVsp") and checkIfExists(rootDataFile, "hdEdxTotVsP_Pos_TPC"):
    canvas = ROOT.TCanvas("","",1200,600)
    legend = ROOT.TLegend()
    profilesAllRuns = [getPIDProfiles(file,charge="pos",rebin=4)[1] for file in rootDataFile]
    canvas.cd()
    for i,run in enumerate(runList):
        for j in range(len(profilesAllRuns[0])):
            profilesAllRuns[i][j].SetName(f"{run}-{profilesAllRuns[i][j].GetName()}")
            profilesAllRuns[i][j].SetMinimum(20)
            profilesAllRuns[i][j].SetMaximum(6000)
            profilesAllRuns[i][j].SetLineWidth(2)
            profilesAllRuns[i][j].SetLineColor(colors[i])
            profilesAllRuns[i][j].Draw("HIST E SAME")
        #legend.AddEntry(profilesAllRuns[i][j],run)
        legend.AddEntry(profilesAllRuns[i][j],LegRunParam[i])
    
    # Loop over profiles (j) for all runs
    #for j in range(len(profilesAllRuns[0])):  # Iterate over the same plot for all runs
        # For each profile j, compare the same profile across all runs
        histoOut, canvasOut = outlierDetection(f"profile_{j}", [profilesAllRuns[i][j] for i in range(len(runList))], runList)
    
    canvas.SetGrid()
    canvas.SetLogx()
    canvas.SetLogy()
    legend.SetNColumns(math.ceil(len(rootDataFile)/8))
    legend.SetHeader("#Run polarity IRstart IRmid IRend", "C")
    legend.Draw()
    canvas.Draw()

In [ ]:
%jsroot on
if checkIfExists(rootDataFile, "CdEdxPIDHypothesisVsp") and checkIfExists(rootDataFile, "hdEdxTotVsP_Neg_TPC"):
    canvas = ROOT.TCanvas("","",1200,600)
    legend = ROOT.TLegend()
    profilesAllRuns = [getPIDProfiles(file,charge="neg",rebin=4)[1] for file in rootDataFile]
    canvas.cd()
    for i,run in enumerate(runList):
        for j in range(len(profilesAllRuns[0])):
            profilesAllRuns[i][j].SetName(f"{run}-{profilesAllRuns[i][j].GetName()}")
            profilesAllRuns[i][j].SetMinimum(20)
            profilesAllRuns[i][j].SetMaximum(6000)
            profilesAllRuns[i][j].SetLineWidth(2)
            profilesAllRuns[i][j].SetLineColor(colors[i])
            profilesAllRuns[i][j].Draw("HIST E SAME")
        #legend.AddEntry(profilesAllRuns[i][j],run)
        legend.AddEntry(profilesAllRuns[i][j],LegRunParam[i])
    
    # Loop over profiles (j) for all runs
    for j in range(len(profilesAllRuns[0])):  # Iterate over the same plot for all runs
        # For each profile j, compare the same profile across all runs
        histoOut, canvasOut = outlierDetection(f"profile_{j}", [profilesAllRuns[i][j] for i in range(len(runList))], runList)
    
    canvas.SetGrid()
    canvas.SetLogx()
    canvas.SetLogy()
    legend.SetNColumns(math.ceil(len(rootDataFile)/8))
    legend.SetHeader("#Run polarity IRstart IRmid IRend", "C")
    legend.Draw()
    canvas.Draw()